In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_reviews").getOrCreate()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, DateType, StructType

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", IntegerType(), True),StructField("review_id", StringType(), True),StructField("product_id", StringType(), True),StructField("product_parent", IntegerType(), True),StructField("product_title", StringType(), True),StructField("product_category", StringType(), True),StructField("star_rating", IntegerType(), True),StructField("helpful_votes", IntegerType(), True),StructField("total_votes", IntegerType(), True),StructField("vine", StringType(), True),StructField("verified_purchase", StringType(), True),StructField("review_headline", StringType(), True),StructField("review_body", StringType(), True),StructField("review_date", DateType(), True)]
schema

[StructField(marketplace,StringType,true),
 StructField(customer_id,IntegerType,true),
 StructField(review_id,StringType,true),
 StructField(product_id,StringType,true),
 StructField(product_parent,IntegerType,true),
 StructField(product_title,StringType,true),
 StructField(product_category,StringType,true),
 StructField(star_rating,IntegerType,true),
 StructField(helpful_votes,IntegerType,true),
 StructField(total_votes,IntegerType,true),
 StructField(vine,StringType,true),
 StructField(verified_purchase,StringType,true),
 StructField(review_headline,StringType,true),
 StructField(review_body,StringType,true),
 StructField(review_date,DateType,true)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,DateType,true)))

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
electronics_mod_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), schema=final, sep="\t", header=True)

electronics_mod_data_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [ ]:
electronics_mod_data_df.describe()

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string]

In [ ]:
electronics_mod_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Count the number of records (rows) in the dataset
electronics_mod_data_df.count()

3093869

In [ ]:
# Drop any rows with null or “not a number” (NaN) values
electronics_clean_data_df = electronics_mod_data_df.dropna()
electronics_clean_data_df.count()

3093660

In [ ]:
electronics_clean_data_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Create user dataframe to match "vine_table" table
cleaned_vine_df = electronics_clean_data_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
cleaned_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Create the dataframe where "vine" is "Y" to verify whether it belong to Amazon's "vine-program" or not 
Yes_vine_df = cleaned_vine_df.filter(col("vine") == "Y") 
Yes_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1T7X1RPWYTRBU|          5|            2|          4|   Y|
| RJFGZBIFXCC65|          5|            0|          0|   Y|
| RSZ6RLTTGWUN6|          5|            0|          1|   Y|
|R2N4GVP3Q5PF9Y|          3|            2|          3|   Y|
| RCAFL1X52MFZM|          5|            1|          2|   Y|
|R17D8E4KO0FS83|          5|            1|          1|   Y|
|R184FOUNZZ7KO8|          5|           15|         20|   Y|
|R2SP52AJMIERZ5|          3|            0|          0|   Y|
|R15VB3EZ2C0W3J|          4|            0|          0|   Y|
|R3CMY924EQEGIU|          4|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
Yes_vine_df.count()

18512

In [ ]:
# Count the star_ratings
ratings_Yes_vine = Yes_vine_df.groupBy(Yes_vine_df["star_rating"]).count()
ratings_Yes_vine.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|  342|
|          3| 2656|
|          5| 8044|
|          4| 6539|
|          2|  931|
+-----------+-----+



In [ ]:
from pyspark.sql.functions import mean 

# Count the average ratings
avg_Yes_vine = Yes_vine_df.select(mean(col("star_rating")))
avg_Yes_vine.show()

+----------------+
|avg(star_rating)|
+----------------+
|4.13504753673293|
+----------------+



In [ ]:
from pyspark.sql.functions import avg
Yes_vine_df.select(avg("star_rating")).show()

+----------------+
|avg(star_rating)|
+----------------+
|4.13504753673293|
+----------------+



In [ ]:
# Count the helpful_votes
helpful_Yes_vine = Yes_vine_df.groupBy(Yes_vine_df["helpful_votes"]).count()
helpful_Yes_vine.show()

+-------------+-----+
|helpful_votes|count|
+-------------+-----+
|          148|    1|
|          243|    1|
|          540|    1|
|           31|   18|
|           85|    4|
|           65|    2|
|           53|    8|
|           78|    3|
|          321|    1|
|          155|    2|
|          108|    2|
|           34|   16|
|          211|    2|
|          115|    3|
|          126|    1|
|           81|    3|
|           28|   18|
|           76|    3|
|           26|   31|
|           27|   31|
+-------------+-----+
only showing top 20 rows



In [ ]:
# Create the dataframe where "vine" is "N" to verify whether it belong to Amazon's "vine-program" or not 
NO_vine_df = cleaned_vine_df.filter(col("vine") == "N") 
NO_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
NO_vine_df.count()

3075148

In [ ]:
# Count the star_ratings
ratings_No_vine = NO_vine_df.groupBy(NO_vine_df["star_rating"]).count()
ratings_No_vine.show()

+-----------+-------+
|star_rating|  count|
+-----------+-------+
|          1| 357743|
|          3| 235919|
|          5|1772987|
|          4| 530258|
|          2| 178241|
+-----------+-------+



In [ ]:
# Count the helpful_votes
helpful_No_vine = NO_vine_df.groupBy(NO_vine_df["helpful_votes"]).count()
helpful_No_vine.show()

+-------------+-----+
|helpful_votes|count|
+-------------+-----+
|          148|   25|
|         1088|    1|
|         3794|    1|
|         1591|    1|
|          471|    1|
|          496|    1|
|          243|   12|
|         1395|    1|
|          392|    3|
|         1721|    1|
|          540|    1|
|         1084|    1|
|          623|    1|
|           31|  964|
|          516|    1|
|           85|  123|
|          137|   33|
|          251|    8|
|          451|    4|
|          580|    1|
+-------------+-----+
only showing top 20 rows



In [ ]:
# Count the average ratings
avg_No_vine = NO_vine_df.select(mean(col("star_rating")))
avg_No_vine.show()

+-----------------+
| avg(star_rating)|
+-----------------+
|4.034911165251233|
+-----------------+

